In [1]:
import sys
sys.path.append('../../ml_utils')

import config as cfg
import data_utils as du
import autoencoder as aenc
import numpy as np
import pandas as pd
import timeit
import matplotlib.pyplot as plt
import joblib
import sklearn

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from keras import regularizers

%matplotlib inline

Using TensorFlow backend.
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
print("joblib (0.14.1): {}\n".format(joblib.__version__))
print("numpy (1.17.4): {}\n".format(np.__version__))
print("pandas (0.25.3): {}\n".format(pd.__version__))
print("sklearn (0.22): {}\n".format(sklearn.__version__))

joblib (0.14.1): 0.14.1

numpy (1.17.4): 1.17.4

pandas (0.25.3): 0.25.3

sklearn (0.22): 0.22



### Load and clean the data

In [3]:
log_data = './../../shared/data/swissid_authorize_logs_april_to_sept_2019.csv'

start = timeit.default_timer()
df = pd.read_csv(filepath_or_buffer=log_data, header=0, sep='\t', names=cfg.complete_feature_list, index_col=None)
stop = timeit.default_timer()

print("Original data frame size: {}\n".format(df.shape))
print("Time: {} seconds\n".format(stop - start))

Original data frame size: (12417597, 31)

Time: 39.89783512299982 seconds



In [4]:
df = du.clean_data(df)

columns with NaN: loc_country_code
loc_country_code is fixed


In [5]:
features = ['label_nr', 'src_software_sub_type', 'src_operating_system_name', 'src_hardware_type', 'response_status_code', 
            'oidc_client_id', 'oidc_scopes', 'oidc_ui_locales', 'loc_city', 'loc_country_code', 'date_weekday']

excludes = list(set(df.columns.tolist()) - set(features))

In [6]:
reduced_df = du.reduce_features(df, excludes)

print("reduced data frame size: {}\n".format(reduced_df.shape))
#display(reduced_df)

reduced data frame size: (12204748, 11)



In [7]:
print(reduced_df['label_nr'].value_counts())

2    7412813
0    4769169
1      22766
Name: label_nr, dtype: int64


### Build a sampled dataframe having the same anomaly rate as the original dateset and collect the metrics data


In [8]:
iterations = 10
metrics = []
iter_values = []

In [9]:
sample_size= 200000
exclude = ['label_nr']

batch_size = 32
epochs = 20
optimizer='Adamax'
encoding_dim = 4

start_overall = timeit.default_timer()

for i in range(iterations):

    X_df, y_df, anomaly_rate, encoder = du.build_normal_anomaly_sample_df(reduced_df, sample_size, exclude, True, True, False, True)
    x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=42)

    
    #models
    sparse_l1 = aenc.create_sparse_auto_encoder(x_train.shape[1], encoding_dim)
    sparse_l2 = aenc.create_sparse_auto_encoder(x_train.shape[1], encoding_dim, 'relu', regularizers.l2(10e-5))
    
    deep_l1 = aenc.create_deep_auto_encoder_2(x_train.shape[1], 7, encoding_dim)
    deep_l2 = aenc.create_deep_auto_encoder_2(x_train.shape[1], 7, encoding_dim, 'relu', regularizers.l2(10e-5))
    
    start_fitting = timeit.default_timer()
    history = aenc.auto_encoder_fit(sparse_l1, x_train, x_test, optimizer, epochs, batch_size)
    history = aenc.auto_encoder_fit(sparse_l2, x_train, x_test, optimizer, epochs, batch_size)
    history = aenc.auto_encoder_fit(deep_l1, x_train, x_test, optimizer, epochs, batch_size)
    history = aenc.auto_encoder_fit(deep_l2, x_train, x_test, optimizer, epochs, batch_size)
    stop_fitting = timeit.default_timer()
    
    fit_time = stop_fitting - start_fitting
    print("{0} Time to fit: {1:.2f} seconds".format(i, fit_time))
    
    
    score_s1 = sparse_l1.evaluate(x_test, x_test, verbose=1)
    score_s2 = sparse_l2.evaluate(x_test, x_test, verbose=1)
    score_d1 = deep_l1.evaluate(x_test, x_test, verbose=1)
    score_d2 = deep_l2.evaluate(x_test, x_test, verbose=1)
    
    iter_values = [score_s1[0],score_s1[1],score_s1[2],
                   score_s2[0],score_s2[1],score_s2[2],
                   score_d1[0],score_d1[1],score_d1[2],
                   score_d2[0],score_d2[1],score_d2[2]
                  ]
        
    metrics.append(iter_values)
    
    du.create_autoencoder_selection_metric_df(metrics, './autoencoder_selection_metrics.csv')
    
    iter_values = []

stop_overall = timeit.default_timer()

overall_runtime = stop_overall - start_overall
print("Time: {0:.2f} seconds\n".format(overall_runtime))
print("Time: {0:.2f} minutes\n".format(overall_runtime/60))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 140000 samples, validate on 60000 samples
Epoch 1/20
140000/140000 [==============================] - 14s 97us/step - loss: 0.9062 - mean_absolute_error: 0.5014 - acc: 0.4661 - val_loss: 0.8865 - val_mean_absolute_error: 0.4822 - val_acc: 0.6504
Epoch 2/20
140000/140000 [==============================] - 13s 94us/step - loss: 0.8678 - mean_absolute_error: 0.4786 - acc: 0.6335 - val_loss: 0.8728 - val_mean_absolute_error: 0.4783 - val_acc: 0.6714
Epoch 3/20
140000/140000 [==============================] - 13s 93us/step - loss: 0.8539 - mean_absolute_error: 0.4742 - acc: 0.7591 - val_loss: 0.8624 - val_mean_absolute_error: 0.4732 - val_acc: 0.7804
Epoch 4/20
140000/140000 [==============================] - 13s 94us/step - loss: 0.8487 - mean_absolute_error: 0.4711 - acc: 0.7958 - val_loss: 0.8601 - val_mean_absolute_error: 0.4719 - val_acc: 0.8028
Epoch 5/20
14

140000/140000 [==============================] - 13s 95us/step - loss: 0.8384 - mean_absolute_error: 0.4695 - acc: 0.8176 - val_loss: 0.8507 - val_mean_absolute_error: 0.4707 - val_acc: 0.8389
Epoch 18/20
140000/140000 [==============================] - 13s 95us/step - loss: 0.8383 - mean_absolute_error: 0.4694 - acc: 0.8202 - val_loss: 0.8506 - val_mean_absolute_error: 0.4707 - val_acc: 0.8118
Epoch 19/20
140000/140000 [==============================] - 13s 94us/step - loss: 0.8382 - mean_absolute_error: 0.4694 - acc: 0.8239 - val_loss: 0.8505 - val_mean_absolute_error: 0.4706 - val_acc: 0.8489
Epoch 20/20
140000/140000 [==============================] - 13s 94us/step - loss: 0.8381 - mean_absolute_error: 0.4694 - acc: 0.8285 - val_loss: 0.8504 - val_mean_absolute_error: 0.4707 - val_acc: 0.8177
Train on 140000 samples, validate on 60000 samples
Epoch 1/20
140000/140000 [==============================] - 17s 125us/step - loss: 0.8946 - mean_absolute_error: 0.5022 - acc: 0.6045 - val_l

140000/140000 [==============================] - 18s 125us/step - loss: 0.8526 - mean_absolute_error: 0.4744 - acc: 0.6961 - val_loss: 0.8648 - val_mean_absolute_error: 0.4755 - val_acc: 0.6686
Epoch 17/20
140000/140000 [==============================] - 18s 126us/step - loss: 0.8524 - mean_absolute_error: 0.4743 - acc: 0.6963 - val_loss: 0.8645 - val_mean_absolute_error: 0.4754 - val_acc: 0.7232
Epoch 18/20
140000/140000 [==============================] - 17s 124us/step - loss: 0.8523 - mean_absolute_error: 0.4742 - acc: 0.7019 - val_loss: 0.8643 - val_mean_absolute_error: 0.4754 - val_acc: 0.6958
Epoch 19/20
140000/140000 [==============================] - 17s 123us/step - loss: 0.8521 - mean_absolute_error: 0.4741 - acc: 0.7004 - val_loss: 0.8642 - val_mean_absolute_error: 0.4755 - val_acc: 0.7229
Epoch 20/20
140000/140000 [==============================] - 17s 124us/step - loss: 0.8520 - mean_absolute_error: 0.4741 - acc: 0.7000 - val_loss: 0.8643 - val_mean_absolute_error: 0.4753 

140000/140000 [==============================] - 14s 98us/step - loss: 0.8680 - mean_absolute_error: 0.4768 - acc: 0.7239 - val_loss: 0.8413 - val_mean_absolute_error: 0.4744 - val_acc: 0.7324
Epoch 15/20
140000/140000 [==============================] - 14s 99us/step - loss: 0.8678 - mean_absolute_error: 0.4767 - acc: 0.7216 - val_loss: 0.8411 - val_mean_absolute_error: 0.4743 - val_acc: 0.7278
Epoch 16/20
140000/140000 [==============================] - 14s 99us/step - loss: 0.8676 - mean_absolute_error: 0.4767 - acc: 0.7206 - val_loss: 0.8410 - val_mean_absolute_error: 0.4743 - val_acc: 0.7237
Epoch 17/20
140000/140000 [==============================] - 14s 98us/step - loss: 0.8674 - mean_absolute_error: 0.4767 - acc: 0.7208 - val_loss: 0.8408 - val_mean_absolute_error: 0.4742 - val_acc: 0.7288
Epoch 18/20
140000/140000 [==============================] - 14s 98us/step - loss: 0.8673 - mean_absolute_error: 0.4766 - acc: 0.7212 - val_loss: 0.8407 - val_mean_absolute_error: 0.4742 - val

140000/140000 [==============================] - 18s 132us/step - loss: 0.8382 - mean_absolute_error: 0.4751 - acc: 0.7378 - val_loss: 0.8122 - val_mean_absolute_error: 0.4727 - val_acc: 0.7042
Epoch 14/20
140000/140000 [==============================] - 19s 132us/step - loss: 0.8380 - mean_absolute_error: 0.4750 - acc: 0.7381 - val_loss: 0.8119 - val_mean_absolute_error: 0.4727 - val_acc: 0.7151
Epoch 15/20
140000/140000 [==============================] - 19s 132us/step - loss: 0.8377 - mean_absolute_error: 0.4750 - acc: 0.7359 - val_loss: 0.8117 - val_mean_absolute_error: 0.4727 - val_acc: 0.7426
Epoch 16/20
140000/140000 [==============================] - 18s 132us/step - loss: 0.8376 - mean_absolute_error: 0.4749 - acc: 0.7340 - val_loss: 0.8116 - val_mean_absolute_error: 0.4726 - val_acc: 0.7388
Epoch 17/20
140000/140000 [==============================] - 18s 132us/step - loss: 0.8374 - mean_absolute_error: 0.4749 - acc: 0.7306 - val_loss: 0.8115 - val_mean_absolute_error: 0.4727 

...

140000/140000 [==============================] - 21s 147us/step - loss: 0.8314 - mean_absolute_error: 0.4701 - acc: 0.8924 - val_loss: 0.8023 - val_mean_absolute_error: 0.4685 - val_acc: 0.9187
Epoch 16/20
140000/140000 [==============================] - 21s 147us/step - loss: 0.8312 - mean_absolute_error: 0.4701 - acc: 0.8937 - val_loss: 0.8026 - val_mean_absolute_error: 0.4684 - val_acc: 0.8925
Epoch 17/20
140000/140000 [==============================] - 21s 147us/step - loss: 0.8310 - mean_absolute_error: 0.4701 - acc: 0.8973 - val_loss: 0.8019 - val_mean_absolute_error: 0.4682 - val_acc: 0.9198
Epoch 18/20
140000/140000 [==============================] - 21s 148us/step - loss: 0.8306 - mean_absolute_error: 0.4700 - acc: 0.8955 - val_loss: 0.8014 - val_mean_absolute_error: 0.4684 - val_acc: 0.9075
Epoch 19/20
140000/140000 [==============================] - 21s 147us/step - loss: 0.8304 - mean_absolute_error: 0.4698 - acc: 0.8990 - val_loss: 0.8012 - val_mean_absolute_error: 0.4681 

140000/140000 [==============================] - 17s 120us/step - loss: 0.8295 - mean_absolute_error: 0.4726 - acc: 0.9046 - val_loss: 0.8277 - val_mean_absolute_error: 0.4711 - val_acc: 0.8972
Epoch 14/20
140000/140000 [==============================] - 17s 120us/step - loss: 0.8292 - mean_absolute_error: 0.4726 - acc: 0.9040 - val_loss: 0.8274 - val_mean_absolute_error: 0.4711 - val_acc: 0.9049
Epoch 15/20
140000/140000 [==============================] - 17s 120us/step - loss: 0.8290 - mean_absolute_error: 0.4726 - acc: 0.9046 - val_loss: 0.8273 - val_mean_absolute_error: 0.4710 - val_acc: 0.8958
Epoch 16/20
140000/140000 [==============================] - 17s 119us/step - loss: 0.8287 - mean_absolute_error: 0.4726 - acc: 0.9060 - val_loss: 0.8270 - val_mean_absolute_error: 0.4710 - val_acc: 0.9120
Epoch 17/20
140000/140000 [==============================] - 17s 119us/step - loss: 0.8285 - mean_absolute_error: 0.4725 - acc: 0.9054 - val_loss: 0.8268 - val_mean_absolute_error: 0.4711 

140000/140000 [==============================] - 21s 150us/step - loss: 0.8367 - mean_absolute_error: 0.4779 - acc: 0.7550 - val_loss: 0.8344 - val_mean_absolute_error: 0.4763 - val_acc: 0.7724
Epoch 13/20
140000/140000 [==============================] - 21s 150us/step - loss: 0.8365 - mean_absolute_error: 0.4779 - acc: 0.7528 - val_loss: 0.8343 - val_mean_absolute_error: 0.4760 - val_acc: 0.7209
Epoch 14/20
140000/140000 [==============================] - 21s 150us/step - loss: 0.8363 - mean_absolute_error: 0.4778 - acc: 0.7573 - val_loss: 0.8341 - val_mean_absolute_error: 0.4762 - val_acc: 0.7732
Epoch 15/20
140000/140000 [==============================] - 21s 150us/step - loss: 0.8362 - mean_absolute_error: 0.4778 - acc: 0.7519 - val_loss: 0.8339 - val_mean_absolute_error: 0.4761 - val_acc: 0.7719
Epoch 16/20
140000/140000 [==============================] - 21s 150us/step - loss: 0.8360 - mean_absolute_error: 0.4778 - acc: 0.7610 - val_loss: 0.8339 - val_mean_absolute_error: 0.4762 

140000/140000 [==============================] - 17s 122us/step - loss: 0.8308 - mean_absolute_error: 0.4758 - acc: 0.8849 - val_loss: 0.8256 - val_mean_absolute_error: 0.4765 - val_acc: 0.8841
Epoch 11/20
140000/140000 [==============================] - 17s 122us/step - loss: 0.8298 - mean_absolute_error: 0.4757 - acc: 0.8902 - val_loss: 0.8247 - val_mean_absolute_error: 0.4763 - val_acc: 0.8875
Epoch 12/20
140000/140000 [==============================] - 17s 122us/step - loss: 0.8289 - mean_absolute_error: 0.4755 - acc: 0.8884 - val_loss: 0.8239 - val_mean_absolute_error: 0.4761 - val_acc: 0.8940
Epoch 13/20
140000/140000 [==============================] - 17s 122us/step - loss: 0.8282 - mean_absolute_error: 0.4753 - acc: 0.8919 - val_loss: 0.8232 - val_mean_absolute_error: 0.4759 - val_acc: 0.8891
Epoch 14/20
140000/140000 [==============================] - 17s 121us/step - loss: 0.8275 - mean_absolute_error: 0.4751 - acc: 0.8939 - val_loss: 0.8226 - val_mean_absolute_error: 0.4757 

140000/140000 [==============================] - 21s 154us/step - loss: 0.8469 - mean_absolute_error: 0.4755 - acc: 0.7583 - val_loss: 0.8424 - val_mean_absolute_error: 0.4765 - val_acc: 0.7538
Epoch 10/20
140000/140000 [==============================] - 21s 153us/step - loss: 0.8466 - mean_absolute_error: 0.4754 - acc: 0.7605 - val_loss: 0.8422 - val_mean_absolute_error: 0.4761 - val_acc: 0.7688
Epoch 11/20
140000/140000 [==============================] - 21s 152us/step - loss: 0.8431 - mean_absolute_error: 0.4737 - acc: 0.7796 - val_loss: 0.8258 - val_mean_absolute_error: 0.4716 - val_acc: 0.8271
Epoch 12/20
140000/140000 [==============================] - 21s 153us/step - loss: 0.8280 - mean_absolute_error: 0.4702 - acc: 0.8623 - val_loss: 0.8222 - val_mean_absolute_error: 0.4705 - val_acc: 0.8766
Epoch 13/20
140000/140000 [==============================] - 21s 153us/step - loss: 0.8266 - mean_absolute_error: 0.4699 - acc: 0.8850 - val_loss: 0.8215 - val_mean_absolute_error: 0.4704 

### Calculate Metrics

In [10]:
metrics_df = du.create_autoencoder_selection_metric_df(metrics, './autoencoder_selection_metrics.csv')
metrics_df.describe()

,loss_s1,mae_s1,acc_s1,loss_s2,mae_s2,acc_s2,loss_d1,mae_d1,acc_d1,loss_d2,mae_d2,acc_d2
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.826915,0.471646,0.871488,0.832505,0.472403,0.850135,0.829259,0.472845,0.826658,0.828331,0.472912,0.788935
std,0.017223,0.002551,0.067180,0.016355,0.002460,0.086472,0.021241,0.004611,0.102222,0.024201,0.003877,0.118853
min,0.801649,0.468295,0.759650,0.807183,0.468750,0.725033,0.801047,0.467912,0.649717,0.797380,0.467711,0.531217
25%,0.811292,0.470669,0.816808,0.821347,0.470694,0.783304,0.815419,0.469621,0.752375,0.808181,0.469774,0.724358
50%,0.828581,0.471298,0.889233,0.837148,0.471507,0.855000,0.829813,0.471819,0.857492,0.827889,0.472272,0.815667
75%,0.837907,0.472013,0.915996,0.842650,0.474626,0.931171,0.838834,0.475331,0.907812,0.839713,0.475115,0.879617
max,0.850710,0.476522,0.952567,0.855143,0.475968,0.952050,0.864185,0.481296,0.940300,0.865644,0.478988,0.923600
